<a href="https://colab.research.google.com/github/lagocode/20251125_ForBookConnectChallenge/blob/main/book_recommend_ver1_0_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install streamlit google-generativeai numpy requests beautifulsoup4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 48.4 MB/s eta 0:00:00


In [2]:
pip install -U google-generativeai

In [3]:
pip install pandas openpyxl

In [4]:
!npm install localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴
added 22 packages in 5s
⠴
⠴3 packages are looking for funding
⠴  run `npm fund` for details
⠴

In [5]:
import streamlit as st
import google.generativeai as genai
import requests
from bs4 import BeautifulSoup
import json
import numpy as np
from datetime import datetime

In [6]:
import pandas as pd

In [7]:
import re

In [8]:
import urllib

In [30]:
import time

In [9]:
# ==========================================
# 1. 파일 로드 및 설정
# ==========================================
# 코랩의 파일 경로 (예: '/content/파일이름.xlsx')
file_path = "/content/sample_data/book_list_sample.xlsx"

try:
    # 엑셀 파일 읽기
    df = pd.read_excel(file_path)
    print("✅ 데이터 로드 완료")
    print(f"   - 전체 크기: {df.shape}")
except FileNotFoundError:
    print(f"❌ 파일을 찾을 수 없습니다: {file_path}")

✅ 데이터 로드 완료
   - 전체 크기: (49, 20)


In [10]:
# ==========================================
# 2. 열 이름 확인
# ==========================================
print("\n📋 [현재 열 목록]")
print(df.columns.tolist())
print(df.head())


📋 [현재 열 목록]
['도서 대여도서관', '도서대여일', '반납예정일', '반납일', '반납연기일', '자료상태', '등록번호', '서명', '저자', '발행자', '청구기호', '반납도서관', '도서 대여 값', '편권차', '도서 대여현황의 고유값', '부록여부', '지역내상호대차상태', 'ISBN', '반납연기횟수', '반납연기 도서관부호']
   도서 대여도서관                도서대여일                반납예정일                  반납일  \
0    111035  2025/01/01 00:00:00  2025/01/15 23:59:59  2025/02/02 09:30:52   
1    111035  2025/01/01 00:00:00  2025/01/15 23:59:59  2025/02/02 09:30:54   
2    111035  2025/01/01 00:00:00  2025/01/15 23:59:59  2025/01/17 11:46:53   
3    121002  2025/01/01 00:00:00  2025/01/31 23:59:59  2025/01/31 00:00:00   
4    126024  2025/01/01 00:00:00  2025/01/15 23:59:59  2025/01/14 00:00:00   

  반납연기일  자료상태          등록번호                                         서명  \
0   NaN     1  CMA000104296                              사랑 사랑 사랑하는 이유   
1   NaN     1  AMA000148390                                    사랑이 다예요   
2   NaN     1  AMA000001129                                     억새풀. 5   
3   NaN     1  BJL000324055  이번 생에 정

In [11]:

# ==========================================
# 3. 특정 열 누락 (열 삭제)
# ==========================================
# 삭제하고 싶은 열 이름들을 리스트에 적어주세요.
cols_to_drop = ['도서 대여도서관', '도서대여일', '반납예정일', '반납연기일', '발행자', '반납도서관',
                '도서 대여 값', '편권차', '도서 대여현황의 고유값', '부록여부', '지역내상호대차상태', 'ISBN',
                '반납연기횟수', '반납연기 도서관부호']

# errors='ignore': 해당 열이 없어도 에러 없이 넘어감
ndf = df.copy()
ndf = ndf.drop(columns=cols_to_drop, errors='ignore')

print(f"\n🗑️ [열 삭제 완료]")
print(f"   - 삭제 시도한 열: {cols_to_drop}")
print(f"   - 남은 열: {ndf.columns.tolist()}")


🗑️ [열 삭제 완료]
   - 삭제 시도한 열: ['도서 대여도서관', '도서대여일', '반납예정일', '반납연기일', '발행자', '반납도서관', '도서 대여 값', '편권차', '도서 대여현황의 고유값', '부록여부', '지역내상호대차상태', 'ISBN', '반납연기횟수', '반납연기 도서관부호']
   - 남은 열: ['반납일', '자료상태', '등록번호', '서명', '저자', '청구기호']


In [12]:
# ==========================================
# 4. 특정 열의 텍스트 자르기 (조건부 슬라이싱)
# ==========================================
target_column = '청구기호'

def slice_left_3(text):
    """
    . 이나 - 를 찾아 그 인덱스 기준 왼쪽 3글자를 반환하는 함수
    예: '813.6' -> '813', '1234.5' -> '234', 'AB-99' -> 'AB'
    """
    text = str(text)

    # 1. 특수문자 위치 탐색
    idx_dot = text.find('.')
    idx_dash = text.find('-')

    # 2. 유효한 인덱스(찾은 것)만 모으기
    found_indices = [i for i in [idx_dot, idx_dash] if i != -1]

    if found_indices:
        # 3. 가장 먼저 나오는 특수문자 위치 선택
        target_idx = min(found_indices)

        # 4. 해당 위치 기준 왼쪽 3글자 슬라이싱
        # target_idx - 3이 음수면 0(처음)부터 시작
        start_idx = max(0, target_idx - 3)
        return text[start_idx:target_idx]
    else:
        # 5. 특수문자가 없으면 그냥 맨 앞 3글자 반환
        return text[:3]

if target_column in ndf.columns:
    # apply 함수를 사용해 각 셀에 로직 적용
    ndf[target_column] = ndf[target_column].apply(slice_left_3)

    print(f"\n✂️ [글자 수 자르기 완료]")
    print(f"   - 대상 열: '{target_column}'")
    print(f"   - 결과 미리보기:\n{ndf[[target_column]].head()}")
else:
    print(f"\n⚠️ '{target_column}' 열이 데이터에 존재하지 않습니다.")


✂️ [글자 수 자르기 완료]
   - 대상 열: '청구기호'
   - 결과 미리보기:
  청구기호
0  843
1  811
2  813
3  513
4  811


In [13]:
# ==========================================
# 5. 추가 슬라이싱 (숫자 뒤 띄어쓰기 기준 절삭)
# ==========================================
# 적용할 열 이름을 지정하세요 (테스트용으로 '반납일' 사용)
target_column_space = '반납일'

def slice_digit_and_space(text):
    """
    숫자(\d) 다음에 공백(\s)이 오면, 공백부터 그 뒤를 모두 삭제
    예: 'Room 1 A' -> 'Room 1', 'Shelf 22 B' -> 'Shelf 22'
    """
    text = str(text)
    # 정규식: 숫자(\d) 뒤에 공백(\s)이 있는 패턴 검색
    match = re.search(r'\d\s', text)
    if match:
        # match.start()는 숫자의 시작 인덱스
        # 숫자까지는 살려야 하므로 start() + 1 까지 슬라이싱
        return text[:match.start() + 1]
    return text

if target_column_space in df.columns:
    ndf[target_column_space] = ndf[target_column_space].apply(slice_digit_and_space)

    print(f"\n✂️ [슬라이싱 2 완료: 숫자 뒤 공백 기준]")
    print(f"   - 대상 열: '{target_column_space}'")
    print(f"   - 결과 미리보기:\n{ndf[[target_column_space]].head()}")
else:
    print(f"\n⚠️ '{target_column_space}' 열이 데이터에 존재하지 않습니다.")


✂️ [슬라이싱 2 완료: 숫자 뒤 공백 기준]
   - 대상 열: '반납일'
   - 결과 미리보기:
          반납일
0  2025/02/02
1  2025/02/02
2  2025/01/17
3  2025/01/31
4  2025/01/14


<>:9: SyntaxWarning: invalid escape sequence '\d'
<>:9: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipython-input-3201580191.py:9: SyntaxWarning: invalid escape sequence '\d'
  숫자(\d) 다음에 공백(\s)이 오면, 공백부터 그 뒤를 모두 삭제


In [14]:
ndf.head()

,반납일,자료상태,등록번호,서명,저자,청구기호
0,2025/02/02,1,CMA000104296,사랑 사랑 사랑하는 이유,므언 티 반 글 ; 제시카 러브 그림 ; 신형건 옮김,843
1,2025/02/02,1,AMA000148390,사랑이 다예요,김용택 사랑시 선집 ; 김선형 그림,811
2,2025/01/17,1,AMA000001129,억새풀. 5,임선영 저,813
3,2025/01/31,1,BJL000324055,이번 생에 정신과는 처음이라 : 정신과 전문의가 말하는 정신과 사용 설명서,닥터 온실 지음,513
4,2025/01/14,1,BNQ000117704,강(江)이 쓰는 시(詩) : 낙동강 연작 제5 시조집,서태수 지음,811


In [15]:
file_path2 = "/content/sample_data/kdc6list2.xlsx"

try:
    # 엑셀 파일 읽기
    kdc_raw_df = pd.read_excel(file_path2, header=None, names=['기호', '장르명'], dtype={'기호': str})
    print("✅ 데이터 로드 완료")
    print(f"   - 전체 크기: {kdc_raw_df.shape}")
except FileNotFoundError:
    print(f"❌ 파일을 찾을 수 없습니다: {file_path2}")

✅ 데이터 로드 완료
   - 전체 크기: (100, 2)


In [16]:
kdc_raw_df['기호'] = kdc_raw_df['기호'].astype(str)

In [17]:
kdc_raw_df

,기호,장르명
0,000,총류
1,010,도서학 서지학
2,020,문헌정보학
3,030,백과사전
4,040,강연집 수필집 연설문집
...,...,...
95,950,남아메리카
96,960,오세아니아 양극지방
97,970,기타
98,980,지리


In [18]:
import pandas as pd
import re
import os

# ==========================================
# 1. 데이터 준비
# ==========================================

# [A] 사용자의 도서관 대출 데이터 (raw_data)
# library_data = {
#     '반납일': ['2025/02/02', '2025/02/02', '2025/01/17', '2025/01/31', '2025/01/14'],
#     '자료상태': ['1', '1', '1', '1', '1'],
#     '등록번호': ['CMA000104296', 'AMA000148390', 'AMA000001129', 'BJL000324055', 'BNQ000117704'],
#     '서명': ['사랑 사랑 사랑하는 이유', '사랑이 다예요', '억새풀. 5', '이번 생에 정신과는 처음이라 : 정신과 전문의가 말하는 정신과 사용 설명서', '강(江)이 쓰는 시(詩) : 낙동강 연작 제5 시조집'],
#     '저자': ['므언 티 반 글 ; 제시카 러브 그림 ; 신형건 옮김', '김용택 사랑시 선집 ; 김선형 그림', '임선영 저', '닥터 온실 지음', '서태수 지음'],
#     '청구기호': ['843', '811', '813', '513', '811'] # 이미 3자리로 전처리된 상태
# }
df_library = pd.DataFrame(ndf)

# [B] 새로 구한 KDC 장르표 로드 (가상 데이터)
# ★ 실제 사용 시: pd.read_excel('kdc_genre_table.xlsx') 로 교체하세요.
# kdc_raw_data = {
#     '기호': ['000', '100', '200', '300', '400', '500', '510', '513', '800', '810', '811', '813', '840', '843'],
#     '장르명': ['총류', '철학', '종교', '사회과학', '자연과학', '기술과학', '의학', '전기공학/통신', '문학', '한국문학', '한국시', '한국소설', '영미문학', '영미소설']
# }
df_kdc = pd.DataFrame(kdc_raw_df)

# ==========================================
# 2. KDC 장르 매핑 딕셔너리 생성 (Lookup Table)
# ==========================================
# 기호를 Key로, 장르명을 Value로 변환 (검색 속도 향상)
# 예: {'813': '한국소설', '513': '전기공학/통신'}
kdc_dict = dict(zip(df_kdc['기호'].astype(str), df_kdc['장르명']))

def get_genre_name_advanced(call_number, lookup_dict):
    """
    청구기호(3자리)를 이용해 장르표에서 정확한 장르명을 찾습니다.
    (계층적 조회 로직 적용: 정확히 없으면 상위 분류 검색)
    """
    code = str(call_number).split('.')[0] # 소수점 제거 (혹시 있다면)

    # 1. 정확한 매칭 (Exact Match) - 예: 813 -> 한국소설
    if code in lookup_dict:
        return lookup_dict[code]

    # 2. 십단위 매칭 (Tens Fallback) - 예: 813이 없으면 810(한국문학) 검색
    if len(code) == 3:
        tens_code = code[:2] + "0"
        if tens_code in lookup_dict:
            return lookup_dict[tens_code]

    # 3. 백단위 매칭 (Hundreds Fallback) - 예: 813이 없으면 800(문학) 검색
    if len(code) >= 1:
        hundreds_code = code[0] + "00"
        if hundreds_code in lookup_dict:
            return lookup_dict[hundreds_code]

    return "기타" # 매핑 실패 시

# ==========================================
# 3. 데이터 변환 로직 (Helper Functions)
# ==========================================

def clean_author(text):
    """저자명 정제"""
    text = text.split(';')[0]
    text = re.sub(r'(지음|저|글).*', '', text).strip()
    return text

def generate_initial_keywords(title, genre):
    """제목 단어 + 디테일한 장르명으로 키워드 생성"""
    # 제목에서 특수문자 제거하고 단어 단위로 쪼개기
    words = re.sub(r'[^\w\s]', '', title).split()
    # 장르명이 구체적일수록 벡터 품질이 좋아짐 (예: '한국소설')
    keywords = [genre] + words
    return keywords[:6]

# ==========================================
# 4. 실행: 데이터프레임 -> 라이브러리 DB 리스트
# ==========================================
library_db_list = []

print("🔄 데이터 변환 중...")

for idx, row in df_library.iterrows():
    # A. 고급 장르 매핑 적용
    genre_name = get_genre_name_advanced(row['청구기호'], kdc_dict)

    # B. 저자명 정제
    clean_auth = clean_author(row['저자'])

    # C. 날짜 포맷
    loan_date = row['반납일'].replace('/', '-')

    # D. 키워드 생성
    keywords = generate_initial_keywords(row['서명'], genre_name)

    # E. 결과 조립
    book_item = {
        "id": row['등록번호'],
        "title": row['서명'],
        "author": clean_auth,
        "genre": genre_name,  # 이제 '문학' 대신 '영미소설', '한국시' 등이 들어감
        "last_loan_date": loan_date,
        "keywords": keywords
    }
    library_db_list.append(book_item)

# ==========================================
# 5. 결과 출력
# ==========================================
import json
print("\n### 변환 결과 (복사해서 app.py의 LIBRARY_DB에 붙여넣으세요) ###\n")
print(json.dumps(library_db_list, indent=4, ensure_ascii=False))

# 검증용 출력
print(f"\n[검증] 변환 예시:")
print(f"- 원본 청구기호: {df_library.iloc[0]['청구기호']} -> 변환 장르: {library_db_list[0]['genre']}")

🔄 데이터 변환 중...

### 변환 결과 (복사해서 app.py의 LIBRARY_DB에 붙여넣으세요) ###

[
    {
        "id": "CMA000104296",
        "title": "사랑 사랑 사랑하는 이유",
        "author": "므언 티 반",
        "genre": "영미문학",
        "last_loan_date": "2025-02-02",
        "keywords": [
            "영미문학",
            "사랑",
            "사랑",
            "사랑하는",
            "이유"
        ]
    },
    {
        "id": "AMA000148390",
        "title": "사랑이 다예요",
        "author": "김용택 사랑시 선집",
        "genre": "한국문학",
        "last_loan_date": "2025-02-02",
        "keywords": [
            "한국문학",
            "사랑이",
            "다예요"
        ]
    },
    {
        "id": "AMA000001129",
        "title": "억새풀. 5",
        "author": "임선영",
        "genre": "한국문학",
        "last_loan_date": "2025-01-17",
        "keywords": [
            "한국문학",
            "억새풀",
            "5"
        ]
    },
    {
        "id": "BJL000324055",
        "title": "이번 생에 정신과는 처음이라 : 정신과 전문의가 말하는 정신과 사용 설명서",
        "author": "닥터 온실",
        "g

In [ ]:
# @title
import pandas as pd
import os

def process_excel_data(df, drop_cols=None, row_range=None):
    """
    데이터프레임을 받아 열 삭제 및 행 자르기를 수행하는 로직 함수
    """
    try:
        # 1. 특정 열 누락 (Drop Columns)
        if drop_cols:
            # 공백 제거 및 존재하는 컬럼만 필터링
            drop_cols = [c.strip() for c in drop_cols if c.strip()]
            all_columns = df.columns.tolist()

            # 실제 데이터에 있는 컬럼만 삭제 대상
            cols_to_drop = [c for c in drop_cols if c in all_columns]

            if cols_to_drop:
                df = df.drop(columns=cols_to_drop)
                print(f"✅ 삭제 완료된 열: {cols_to_drop}")
            else:
                print("ℹ️ 삭제할 열 이름이 데이터에 없거나 입력되지 않았습니다.")

        # 2. 행 범위 선택 (Row Slicing)
        if row_range:
            start_row, end_row = row_range
            # 유효성 검사 (음수 방지)
            start_row = max(0, start_row)
            # 종료 행이 0이면(입력 안함) 끝까지
            if end_row == 0 or end_row > len(df):
                end_row = len(df)

            df = df.iloc[start_row:end_row]
            print(f"✅ 행 범위 선택: {start_row}행 ~ {end_row}행 (추출된 개수: {len(df)}개)")

        return df

    except Exception as e:
        print(f"❌ 데이터 처리 중 오류 발생: {e}")
        return None

def main():
    print("=== 엑셀 데이터 전처리기 (Interactive Mode) ===")

    # 1. 파일 입력 받기
    while True:
        file_path_input = input("\n📂 1. 엑셀 파일 경로를 입력하세요 (예: data.xlsx): ").strip()
        # 따옴표가 포함된 경우 제거
        file_path = file_path_input.replace('"', '').replace("'", "")

        if os.path.exists(file_path):
            try:
                # 엑셀 파일 로드
                df_original = pd.read_excel(file_path)
                print(f"   -> 파일 로드 성공! (총 {len(df_original)}행, {len(df_original.columns)}열)")
                print(f"   -> 현재 열 목록: {df_original.columns.tolist()}")
                break
            except Exception as e:
                print(f"   ❌ 파일을 읽을 수 없습니다: {e}")
        else:
            print("   ❌ 파일이 존재하지 않습니다. 경로를 다시 확인해주세요.")

    # 2. 삭제할 열 입력 받기
    drop_input = input("\n🗑️ 2. 삭제할 열 이름을 쉼표(,)로 구분하여 입력하세요 (없으면 엔터): ")
    drop_cols_list = []
    if drop_input.strip():
        drop_cols_list = drop_input.split(',')

    # 3. 행 범위 입력 받기
    print("\n✂️ 3. 추출할 행 범위를 입력하세요 (숫자만 입력)")
    try:
        start_input = input("   - 시작 행 번호 (기본값 0): ").strip()
        start_row = int(start_input) if start_input else 0

        end_input = input("   - 종료 행 번호 (기본값 끝까지): ").strip()
        end_row = int(end_input) if end_input else 0 # 0이면 끝까지로 처리

        row_range_tuple = (start_row, end_row)
    except ValueError:
        print("   ⚠️ 숫자가 아니어서 기본값(전체 범위)을 사용합니다.")
        row_range_tuple = (0, 0)

    # 4. 처리 함수 실행
    print("\n🔄 처리를 시작합니다...")
    result_df = process_excel_data(df_original, drop_cols=drop_cols_list, row_range=row_range_tuple)

    # 5. 결과 확인 및 저장
    if result_df is not None:
        print("\n=== 결과 데이터 미리보기 (상위 5개) ===")
        print(result_df.head())

        save_yn = input("\n💾 결과를 파일로 저장하시겠습니까? (y/n): ").lower()
        if save_yn == 'y':
            save_path = input("   - 저장할 파일명 (예: result.xlsx): ").strip()
            if not save_path.endswith('.xlsx'):
                save_path += '.xlsx'
            result_df.to_excel(save_path, index=False)
            print(f"   ✅ 저장 완료: {save_path}")

    print("\n프로그램을 종료합니다.")

if __name__ == "__main__":
    main()

=== 엑셀 데이터 전처리기 (Interactive Mode) ===

📂 1. 엑셀 파일 경로를 입력하세요 (예: data.xlsx): /content/sample_data/Book_1_1.xlsx


KeyboardInterrupt: 

In [ ]:
# @title
# ==========================================
# 0. 데이터베이스 (30권 - 유동적 데이터 가정)
# ==========================================
# 키워드는 사용자의 리뷰나 데이터 업데이트에 따라 변할 수 있다고 가정합니다.
LIBRARY_DB = [
    {"id": 1, "title": "데미안", "author": "헤르만 헤세", "genre": "소설", "last_loan_date": "2023-12-01", "keywords": ["자아 성찰", "성장", "내면의 투쟁", "고전", "우정"]},
    {"id": 2, "title": "1984", "author": "조지 오웰", "genre": "SF", "last_loan_date": "2024-01-15", "keywords": ["디스토피아", "감시", "전체주의", "미래", "정치"]},
    {"id": 3, "title": "멋진 신세계", "author": "올더스 헉슬리", "genre": "SF", "last_loan_date": "2023-11-20", "keywords": ["디스토피아", "과학 기술", "인간성 상실", "미래", "풍자"]},
    {"id": 4, "title": "이방인", "author": "알베르 카뮈", "genre": "소설", "last_loan_date": "2024-02-01", "keywords": ["부조리", "실존주의", "무관심", "살인", "철학"]},
    {"id": 5, "title": "호밀밭의 파수꾼", "author": "J.D. 샐린저", "genre": "소설", "last_loan_date": "2023-10-05", "keywords": ["청소년", "방황", "순수함", "성장", "반항"]},
    {"id": 6, "title": "노르웨이의 숲", "author": "무라카미 하루키", "genre": "로맨스", "last_loan_date": "2024-01-10", "keywords": ["상실", "사랑", "고독", "기억", "청춘"]},
    {"id": 7, "title": "참을 수 없는 존재의 가벼움", "author": "밀란 쿤데라", "genre": "소설", "last_loan_date": "2023-09-15", "keywords": ["사랑", "철학", "역사", "운명", "가벼움"]},
    {"id": 8, "title": "채식주의자", "author": "한강", "genre": "소설", "last_loan_date": "2024-02-10", "keywords": ["폭력", "트라우마", "식물성", "가족", "거부"]},
    {"id": 9, "title": "소년이 온다", "author": "한강", "genre": "역사", "last_loan_date": "2023-12-25", "keywords": ["역사", "광주", "애도", "폭력", "양심"]},
    {"id": 10, "title": "코스모스", "author": "칼 세이건", "genre": "과학", "last_loan_date": "2023-08-01", "keywords": ["우주", "과학", "인류", "기원", "철학"]},
    {"id": 11, "title": "사피엔스", "author": "유발 하라리", "genre": "인문", "last_loan_date": "2024-01-01", "keywords": ["인류사", "진화", "미래", "문명", "과학"]},
    {"id": 12, "title": "총 균 쇠", "author": "제러드 다이아몬드", "genre": "인문", "last_loan_date": "2023-11-01", "keywords": ["문명", "지리", "역사", "불평등", "환경"]},
    {"id": 13, "title": "이기적 유전자", "author": "리처드 도킨스", "genre": "과학", "last_loan_date": "2023-12-10", "keywords": ["진화론", "과학", "유전자", "생물학", "생존"]},
    {"id": 14, "title": "지적 대화를 위한 넓고 얕은 지식", "author": "채사장", "genre": "인문", "last_loan_date": "2024-01-20", "keywords": ["교양", "인문학", "철학", "역사", "경제"]},
    {"id": 15, "title": "돈의 속성", "author": "김승호", "genre": "경제", "last_loan_date": "2024-02-15", "keywords": ["경제", "부자", "투자", "철학", "돈"]},
    {"id": 16, "title": "부자 아빠 가난한 아빠", "author": "로버트 기요사키", "genre": "경제", "last_loan_date": "2023-10-30", "keywords": ["재테크", "금융", "경제 관념", "자산", "투자"]},
    {"id": 17, "title": "해리 포터와 마법사의 돌", "author": "J.K. 롤링", "genre": "판타지", "last_loan_date": "2024-01-05", "keywords": ["판타지", "마법", "모험", "우정", "성장"]},
    {"id": 18, "title": "반지의 제왕", "author": "J.R.R. 톨킨", "genre": "판타지", "last_loan_date": "2023-09-01", "keywords": ["판타지", "서사시", "선악 대결", "모험", "신화"]},
    {"id": 19, "title": "듄", "author": "프랭크 허버트", "genre": "SF", "last_loan_date": "2023-12-20", "keywords": ["SF", "정치", "종교", "생태", "권력"]},
    {"id": 20, "title": "나미야 잡화점의 기적", "author": "히가시노 게이고", "genre": "소설", "last_loan_date": "2024-02-05", "keywords": ["감동", "미스터리", "치유", "편지", "인연"]},
    {"id": 21, "title": "용의자 X의 헌신", "author": "히가시노 게이고", "genre": "추리", "last_loan_date": "2023-11-15", "keywords": ["추리", "사랑", "두뇌 싸움", "희생", "수학"]},
    {"id": 22, "title": "아몬드", "author": "손원평", "genre": "소설", "last_loan_date": "2024-01-25", "keywords": ["성장", "공감", "청소년", "감정", "가족"]},
    {"id": 23, "title": "불편한 편의점", "author": "김호연", "genre": "소설", "last_loan_date": "2024-02-18", "keywords": ["위로", "이웃", "일상", "편의점", "회복"]},
    {"id": 24, "title": "달러구트 꿈 백화점", "author": "이미예", "genre": "판타지", "last_loan_date": "2023-12-15", "keywords": ["판타지", "힐링", "꿈", "무의식", "위로"]},
    {"id": 25, "title": "물고기는 존재하지 않는다", "author": "룰루 밀러", "genre": "에세이", "last_loan_date": "2023-11-10", "keywords": ["과학", "에세이", "삶의 의미", "혼돈", "질서"]},
    {"id": 26, "title": "침묵의 봄", "author": "레이첼 카슨", "genre": "과학", "last_loan_date": "2023-07-20", "keywords": ["환경", "생태", "경고", "자연", "화학"]},
    {"id": 27, "title": "정의란 무엇인가", "author": "마이클 샌델", "genre": "인문", "last_loan_date": "2023-08-15", "keywords": ["철학", "윤리", "사회", "정의", "딜레마"]},
    {"id": 28, "title": "미움받을 용기", "author": "기시미 이치로", "genre": "인문", "last_loan_date": "2024-01-30", "keywords": ["심리학", "아들러", "관계", "용기", "행복"]},
    {"id": 29, "title": "작별하지 않는다", "author": "한강", "genre": "소설", "last_loan_date": "2023-12-30", "keywords": ["제주", "역사", "기억", "고통", "사랑"]},
    {"id": 30, "title": "파친코", "author": "이민진", "genre": "소설", "last_loan_date": "2024-02-12", "keywords": ["이민자", "가족", "역사", "생존", "차별"]},
]

In [ ]:
# ==========================================
# 0. 설정 및 모델 초기화 (가장 중요)
# ==========================================
try:
    from google.colab import userdata
    API_KEY = userdata.get('GEMINI_API_KEY')
except:
    API_KEY = ""

if not API_KEY:
    # 실행 시 입력받거나 여기에 직접 문자열로 입력하세요
    API_KEY = input("Google Gemini API Key를 입력하세요: ").strip()

genai.configure(api_key=API_KEY)

# [수정] model 객체를 전역 변수로 확실하게 생성
try:
    model = genai.GenerativeModel('gemini-2.0-flash')
except Exception as e:
    print(f"⚠️ 모델 초기화 경고: {e}")
    # 구형 라이브러리 대비 Fallback
    model = genai.GenerativeModel('gemini-pro')

In [23]:
# ==========================================
# 2. Google Books API (알라딘 대체)
# ==========================================

def fetch_google_books_description(book_title):
    """
    Google Books API를 이용해 책의 상세 정보를 가져옵니다. (크롤링 아님, API 사용)
    """
    print(f"\n[Google Books] '{book_title}' 정보 검색 중...")

    # Google Books API 엔드포인트
    api_url = "https://www.googleapis.com/books/v1/volumes"
    params = {
        "q": book_title,
        "maxResults": 1,
        "printType": "books",
        "langRestrict": "ko" # 한국어 결과 우선
    }

    try:
        res = requests.get(api_url, params=params)
        data = res.json()

        if "items" in data and len(data["items"]) > 0:
            book_info = data["items"][0]["volumeInfo"]
            title = book_info.get("title", "제목 없음")
            desc = book_info.get("description", "줄거리 정보가 없습니다.")
            authors = ", ".join(book_info.get("authors", []))

            print(f" -> 찾은 책: {title} ({authors})")
            return desc
        else:
            print(" -> Google Books 결과가 없습니다.")
            return None

    except Exception as e:
        print(f" -> API 오류: {e}")
        return None

def llm_extract_keywords_from_description(model, book_title, description):
    """
    책 줄거리(Description)를 LLM에게 주고 핵심 키워드 추출
    """
    print(f"[LLM Thinking] '{book_title}'의 줄거리를 분석해 키워드 추출 중...")

    content_text = description if description else f"{book_title} (줄거리 정보 없음, 일반적 지식 활용)"

    prompt = f"""
    책 제목: {book_title}
    책 줄거리/설명:
    {content_text}

    위 내용을 바탕으로 이 책의 특징을 가장 잘 설명하는 '핵심 키워드' 5개를 추출해줘.
    형용사나 명사 형태로.

    결과는 JSON 리스트로 줘: ["키워드1", "키워드2", "키워드3", "키워드4", "키워드5"]
    """

    try:
        res = model.generate_content(prompt, generation_config={"response_mime_type": "application/json"})
        return json.loads(res.text)
    except Exception as e:
        print(f" -> 키워드 추출 실패: {e}")
        return [book_title, "정보없음", "일반", "도서", "분석불가"]

# ==========================================
# 3. 벡터 및 유사도 함수
# ==========================================

def get_embedding(text):
    try:
        result = genai.embed_content(
            model="models/text-embedding-004",
            content=text,
            task_type="semantic_similarity"
        )
        return np.array(result['embedding'])
    except Exception as e:
        print(f"❌ 임베딩 오류: {e}")
        return np.zeros(768)

def cosine_similarity(vec1, vec2):
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    if norm1 == 0 or norm2 == 0:
        return 0.0
    return np.dot(vec1, vec2) / (norm1 * norm2)

def vectorize_library(library_db):
    print("\n[System] 도서관 보유 장서(30권) 벡터화 시작...")
    vectors = {}
    for i, book in enumerate(library_db):
        print(f"\r -> 진행률: {i+1}/{len(library_db)} ({book['title']})", end="")

        semantic_text = f"장르: {book['genre']}. 핵심 키워드: {', '.join(book['keywords'])}"
        vectors[book['id']] = get_embedding(semantic_text)
        time.sleep(0.1) # Rate Limit 방지
    print("\n[System] 벡터화 완료! ✅")
    return vectors

# ==========================================
# 4. 추천 로직 함수
# ==========================================

def llm_update_user_keywords(model, current_keywords, new_review):
    print(f"\n[LLM Thinking] 사용자 리뷰 분석 중... ('{new_review}')")

    prompt = f"""
    기존 사용자 관심 키워드: {current_keywords}
    새로운 사용자 리뷰: "{new_review}"

    1. 새 리뷰가 기존 키워드에 없는 새로운 취향이나 중요한 통찰을 포함하고 있는지 0~10점으로 평가하세요.
    2. 점수가 7점 이상이라면, 기존 키워드 중 가장 관련성이 떨어지는 것을 제거하고 새 특징을 반영한 키워드를 추가하세요.

    결과는 반드시 JSON 형식으로 주세요:
    {{ "score": 점수(숫자), "reason": "점수 이유", "updated_keywords": ["키워드1", "키워드2", "키워드3", "키워드4", "키워드5"] }}
    """
    try:
        res = model.generate_content(prompt, generation_config={"response_mime_type": "application/json"})
        return json.loads(res.text)
    except Exception as e:
        print(f"LLM Error: {e}")
        return {"score": 0, "reason": "Error", "updated_keywords": current_keywords}

def run_recommendation_logic(user_keywords, current_book_info, library_vectors):
    print(f"\n[Algorithm] 벡터 유사도 계산 및 필터링 시작...")
    print(f" -> 사용자 쿼리: 장르='{current_book_info['genre']}', 키워드={user_keywords}")

    user_query_text = f"장르: {current_book_info['genre']}. 선호 키워드: {', '.join(user_keywords)}"
    user_vector = get_embedding(user_query_text)

    candidates = []

    for book in LIBRARY_DB:
        if book['id'] == current_book_info['id']: continue

        book_vector = library_vectors.get(book['id'])
        score = cosine_similarity(user_vector, book_vector)
        candidates.append({"book": book, "score": score})

    candidates.sort(key=lambda x: x['score'], reverse=True)
    top_10 = candidates[:10]

    print("\n--- [Debug] 유사도 Top 3 후보 ---")
    for i, c in enumerate(top_10[:3]):
        print(f" {i+1}위: {c['book']['title']} ({c['score']:.4f})")

    final_recommendations = []

    # [조건 A] 특수 추천
    same_genre_books = [c for c in top_10 if c['book']['genre'] == current_book_info['genre']]

    if same_genre_books:
        special_pick = same_genre_books[0]
        reason = f"장르({special_pick['book']['genre']})가 같고 유사도가 가장 높음"
    else:
        oldest = sorted(top_10, key=lambda x: x['book']['last_loan_date'])[0]
        special_pick = oldest
        reason = f"장르 매칭 실패 -> 상위권 중 가장 오래된 책({oldest['book']['last_loan_date']})"

    final_recommendations.append({
        "type": "Special Pick",
        "title": special_pick['book']['title'],
        "reason": reason,
        "score": special_pick['score']
    })

    # [조건 B] 일반 추천
    count = 0
    for cand in top_10:
        if cand['book']['id'] == special_pick['book']['id']: continue
        final_recommendations.append({
            "type": "Similarity Pick",
            "title": cand['book']['title'],
            "reason": "단순 벡터 유사도 최상위",
            "score": cand['score']
        })
        count += 1
        if count == 2: break

    return final_recommendations

In [24]:
# json.dumps()는 데이터를 **문자열(String)**로 변환해버리기 때문에 발생하는 에러입니다. Streamlit이나 데이터프레임은 Python 리스트/딕셔너리 객체 그 자체를 필요로 합니다.
# json.dumps를 제거하고 리스트 변수를 그대로 할당하세요.

LIBRARY_DB = json.dumps(library_db_list, indent=4, ensure_ascii=False)

In [26]:
# [수정 전] - 문자열로 변환되어 에러 발생
# LIBRARY_DB = json.dumps(library_db_list, indent=4, ensure_ascii=False)

# [수정 후] - 리스트 객체 그대로 사용 (정상 작동)
LIBRARY_DB = library_db_list

In [31]:
# ==========================================
# 5. 실행 (Main)
# ==========================================
if __name__ == "__main__":
    print("=========================================")
    print("      추천 시스템 로직 검증 모듈")
    print("=========================================")

    # 1. 상황 설정 (책 선택)
    TARGET_BOOK_TITLE = "주식시장을 이긴 전략들:검증된 방법으로 투자하라!"

    # [수정된 부분] 리스트 컴프리헨션으로 안전하게 찾기
    found_books = [b for b in LIBRARY_DB if b['title'] == TARGET_BOOK_TITLE]
    if not found_books:
        print(f"❌ '{TARGET_BOOK_TITLE}' 책을 DB에서 찾을 수 없습니다.")
        exit()
    current_book = found_books[0]

    print(f"\n[Step 0] 선택된 책: {current_book['title']}")
    print(f" -> 초기 키워드: {current_book['keywords']}")

    # 2. [NEW] Google Books API 기반 키워드 추출 로직
    description = fetch_google_books_description(current_book['title'])
    new_extracted_keywords = llm_extract_keywords_from_description(model, current_book['title'], description)

    # 추출된 키워드로 현재 책 정보 업데이트
    current_book['keywords'] = new_extracted_keywords
    print(f" -> 🌟 Google Books/LLM 기반 추출 키워드: {current_book['keywords']}")

    # 3. DB 벡터화
    library_vectors = vectorize_library(LIBRARY_DB)

    # 4. 사용자 후기 입력
    print("\n-----------------------------------------")
    review_input = input("테스트할 후기를 입력하세요 (예: 너무 어렵고 지루했다): ")
    if not review_input: review_input = "초보자가 읽기에는 용어가 너무 어려웠다."

    # 5. 사용자 선호 키워드 업데이트
    update_result = llm_update_user_keywords(model, current_book['keywords'], review_input)

    if update_result['score'] >= 7:
        user_final_keywords = update_result['updated_keywords']
        print(f" -> ✅ 사용자 취향 키워드 변경됨: {user_final_keywords}")
    else:
        user_final_keywords = current_book['keywords']
        print(f" -> ⏹️ 키워드 유지됨: {user_final_keywords}")

    # 6. 최종 추천 로직 실행
    final_recs = run_recommendation_logic(user_final_keywords, current_book, library_vectors)

    # 7. 결과 출력
    print("\n=========================================")
    print("       🎉 최종 추천 결과 (3권)")
    print("=========================================")
    for rec in final_recs:
        print(f"[{rec['type']}] {rec['title']}")
        print(f" - 선정 논리: {rec['reason']}")
        print(f" - 벡터 유사도: {rec['score']:.4f}")
        print("-----------------------------------------")

      추천 시스템 로직 검증 모듈

[Step 0] 선택된 책: 주식시장을 이긴 전략들:검증된 방법으로 투자하라!
 -> 초기 키워드: ['주식시장', '투자전략', '검증된 방법', '투자', '전략']

[Google Books] '주식시장을 이긴 전략들:검증된 방법으로 투자하라!' 정보 검색 중...
 -> 찾은 책: 주식시장을 이긴 전략들(개정판) (박상우)
[LLM Thinking] '주식시장을 이긴 전략들:검증된 방법으로 투자하라!'의 줄거리를 분석해 키워드 추출 중...
 -> 🌟 Google Books/LLM 기반 추출 키워드: ['주식시장', '투자전략', '검증된 방법', '투자', '전략들']

[System] 도서관 보유 장서(30권) 벡터화 시작...
 -> 진행률: 49/49 (부자 아빠 가난한 아빠. 2, '현금흐름 사분면’과 돈을 관리하는 7가지 방법)
[System] 벡터화 완료! ✅

-----------------------------------------
테스트할 후기를 입력하세요 (예: 너무 어렵고 지루했다): 포트폴리오가 중요하다고 생각했고 사회초년생에게 추천합니다

[LLM Thinking] 사용자 리뷰 분석 중... ('포트폴리오가 중요하다고 생각했고 사회초년생에게 추천합니다')
 -> ✅ 사용자 취향 키워드 변경됨: ['주식시장', '투자전략', '포트폴리오', '투자', '사회초년생 투자']

[Algorithm] 벡터 유사도 계산 및 필터링 시작...
 -> 사용자 쿼리: 장르='경제학', 키워드=['주식시장', '투자전략', '포트폴리오', '투자', '사회초년생 투자']

--- [Debug] 유사도 Top 3 후보 ---
 1위: 사랑 사랑 사랑하는 이유 (0.9995)
 2위: 딸에 대하여 : 김혜진 장편소설 (0.9995)
 3위: 한국이 싫어서 : 장강명 장편소설 (0.9995)

       🎉 최종 추천 결과 (3권)
[Special Pick] 딸에 대하여 : 김혜진 장편소설
 - 선정 논리:

In [ ]:
# =========================================
#       추천 시스템 로직 검증 모듈
# =========================================

# [Step 0] 선택된 책: 주식시장을 이긴 전략들:검증된 방법으로 투자하라!
#  -> 초기 키워드: ['주식시장', '투자전략', '검증된 방법', '투자', '전략']

# [Google Books] '주식시장을 이긴 전략들:검증된 방법으로 투자하라!' 정보 검색 중...
#  -> 찾은 책: 주식시장을 이긴 전략들(개정판) (박상우)
# [LLM Thinking] '주식시장을 이긴 전략들:검증된 방법으로 투자하라!'의 줄거리를 분석해 키워드 추출 중...
#  -> 🌟 Google Books/LLM 기반 추출 키워드: ['주식시장', '투자전략', '검증된 방법', '투자', '전략들']

# [System] 도서관 보유 장서(30권) 벡터화 시작...
#  -> 진행률: 49/49 (부자 아빠 가난한 아빠. 2, '현금흐름 사분면’과 돈을 관리하는 7가지 방법)
# [System] 벡터화 완료! ✅

# -----------------------------------------
# 테스트할 후기를 입력하세요 (예: 너무 어렵고 지루했다): 포트폴리오가 중요하다고 생각했고 사회초년생에게 추천합니다

# [LLM Thinking] 사용자 리뷰 분석 중... ('포트폴리오가 중요하다고 생각했고 사회초년생에게 추천합니다')
#  -> ✅ 사용자 취향 키워드 변경됨: ['주식시장', '투자전략', '포트폴리오', '투자', '사회초년생 투자']

# [Algorithm] 벡터 유사도 계산 및 필터링 시작...
#  -> 사용자 쿼리: 장르='경제학', 키워드=['주식시장', '투자전략', '포트폴리오', '투자', '사회초년생 투자']

# --- [Debug] 유사도 Top 3 후보 ---
#  1위: 사랑 사랑 사랑하는 이유 (0.9995)
#  2위: 딸에 대하여 : 김혜진 장편소설 (0.9995)
#  3위: 한국이 싫어서 : 장강명 장편소설 (0.9995)

# =========================================
#        🎉 최종 추천 결과 (3권)
# =========================================
# [Special Pick] 딸에 대하여 : 김혜진 장편소설
#  - 선정 논리: 장르 매칭 실패 -> 상위권 중 가장 오래된 책(2024-10-26)
#  - 벡터 유사도: 0.9995
# -----------------------------------------
# [Similarity Pick] 사랑 사랑 사랑하는 이유
#  - 선정 논리: 단순 벡터 유사도 최상위
#  - 벡터 유사도: 0.9995
# -----------------------------------------
# [Similarity Pick] 한국이 싫어서 : 장강명 장편소설
#  - 선정 논리: 단순 벡터 유사도 최상위
#  - 벡터 유사도: 0.9995
# -----------------------------------------